# Anomaly Detection with FLEXible 

## Import 


In [45]:
from flex.pool import init_server_model
from flex.pool import FlexPool
from flex.model import FlexModel
from flex.pool import deploy_server_model
from flex.data import FedDatasetConfig, FedDataDistribution, Dataset
from app import split_data, load_data_ODDS,define_model_AutoEncoder
import numpy as np 
from copy import deepcopy
import tensorflow as tf


## Load data

In [18]:
data_path = "datasets/shuttle.mat"

In [19]:
def load_data():
    df = load_data_ODDS(data_path)    # shape = (n_samples,2), n_samples = samples number  and 2: X(list of attributes by sample), y (sample label)
    X = np.array(df['X'].tolist())
    y = np.array(df['y'].tolist())
    return (X,y)


## Federated Data 

In [34]:

def federated_data(X,y,n_clients,split_size):
    X_train, X_test, y_train, y_test = split_data(X,y,split_size)
    train_data = Dataset.from_array(X_train, y_train)
    config = FedDatasetConfig(seed=0)
    config.n_nodes = n_clients
    config.replacement = False # ensure that clients do not share any data
    config.node_ids = ["client"+ str(i+1) for i in range(n_clients)] # Optional
    flex_dataset = FedDataDistribution.from_config(centralized_data=train_data, config=config)
    return flex_dataset


## Define, build, deploy and train model

In [46]:
def define_model():
    X,y = load_data()
    input_dim = X.shape[1]
    model =  define_model_AutoEncoder(input_dim)
    return model


@init_server_model
def build_server_model():
    flex_model = FlexModel()
    flex_model["model"] = define_model()
    return flex_model


# @deploy_server_model
# def copy_server_model_to_clients(server_flex_model: FlexModel):
#     return deepcopy(server_flex_model)

@deploy_server_model
def copy_model_to_clients(server_flex_model):
    client_flex_model = FlexModel()
    weights = server_flex_model["model"].get_weights()
    model = tf.keras.models.clone_model(server_flex_model["model"])
    model.set_weights(weights)
    model.compile(
        optimizer=server_flex_model["optimizer"],
        loss=server_flex_model["loss"],
        metrics=server_flex_model["metrics"],
    )
    client_flex_model["model"] = model
    return client_flex_model
    
def train(client_model, client_data):
    print("Training model at client.")
    print(client_model)
    model = client_model['model']
    X_data, y_data = client_data.to_numpy()
    history = model.fit(X_data, y_data)

## Collect Weights

In [38]:
def collect_weights(aggregator_model, clients_model, **kwargs):
    print("Collecting weights.")
    if 'weights' not in aggregator_model:
        aggregator_model['weights'] = []
    for k in clients_model:
        client_weights = clients_model[k]['model'].get_weights()
        aggregator_model['weights'].append(client_weights)

## Aggregate and Set Weights

In [39]:
def aggregate_weights(agg_model, *args):
    print("Aggregating weights")
    averaged_weights = np.mean(np.array(agg_model['weights'], dtype=object), axis=0)
    agg_model["model"].set_weights(averaged_weights)
    agg_model["weights"] = []

In [40]:
def deploy_global_model_to_clients(server_model, clients_models, *args, **kwargs):
    print("Deploying the global model on the clients.")
    aggregated_weights = server_model['model'].get_weights()
    print(clients_models)
    for client_model in clients_models:
        clients_models[client_model]['model'].set_weights(aggregated_weights)

## Evaluate model

In [41]:
def evaluate_model(model, data, *args, **kwargs):
    model = model['model']
    if data is not None:
        print("Evaluating model at client.")
        print(f"Results at client on client's data:")
        model.evaluate(kwargs['test_examples'], kwargs['test_labels'])
        
    else:
        print("Evaluating model at server")
        model.evaluate(kwargs['test_examples'], kwargs['test_labels'])
    




In [48]:
def train_n_rounds(X_test,y_test,flex_dataset,n_rounds):
    pool = FlexPool.client_server_pool(fed_dataset=flex_dataset, init_func=build_server_model)
    pool.servers.map(copy_model_to_clients, pool.clients)
    print(pool.clients)
    for i in range(n_rounds):
        print(f"\nRunning round: {i}\n")
        pool.clients.map(train)
        pool.clients.map(evaluate_model, test_examples=X_test, test_labels=y_test)
        pool.aggregators.map(collect_weights, pool.clients)
        pool.aggregators.map(aggregate_weights)
        pool.servers.map(deploy_global_model_to_clients, pool.clients)
        pool.servers.map(evaluate_model, test_examples=X_test, test_labels=y_test)



In [ ]:

X,y = load_data()
split_size = 0.3
flex_dataset = federated_data(X,y,3,split_size)
X_train, X_test, y_train, y_test = split_data(X,y,split_size)
test_data = Dataset.from_array(X_test, y_test)
train_n_rounds( X_test, y_test,flex_dataset, n_rounds=4)


In [ ]:
p = FlexPool.client_server_pool(flex_dataset, init_func = build_server_model)

clients = p.clients
servers = p.servers
aggregators = p.aggregators

print(f"Number of nodes in the pool {len(p)}: {len(servers)} servers plus {len(clients)} clients. The server is also a aggregators")
print(f"Server node is indentified by {servers.actor_ids}")
print(f"Client nodes are identified by {clients.actor_ids}")
